In [173]:
import nibabel

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob

from pydicom import dcmread
from pathlib import Path
from subprocess import check_call
from tqdm import tqdm
from shutil import rmtree


from mri_brain.contours import mask_to_dicom
from dicom_csv import join_tree
from dicom_csv.spatial import order_series

### LGG dataset creation

1. Populate folders with llg_path.csv - doesn't work because of wrong labels (COR are not in the segmentation)
    1.1 Populate only 38 folders and manually check that they are ok
2. Check if there propeply transferred with 
3. Transfer RT struct
4. Compose rtstruct files into nifty
5. Visually check the dat

In [343]:
basedir + paths['T1'].iloc[i][1:]

'/anvar/public_datasets/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-DU-6395/02-09-1983-BRAINGD-38915/4.000000-ScanningSequence memp-03903'

In [348]:
pd.Series(os.listdir(rtdir))

0     TCGA-FG-6689
1     TCGA-FG-7634
2     TCGA-CS-6188
3     TCGA-HT-7884
4     TCGA-DU-7015
          ...     
61    TCGA-DU-7294
62    TCGA-DU-7304
63    TCGA-FG-A4MT
64    TCGA-HT-8563
65    TCGA-DU-7014
Length: 66, dtype: object

In [352]:
# несмотря на то, что вроде бы 66 данных, нет пересечения с рт структурами, и это странно
len(os.listdir('/anvar/public_datasets/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG'))

67

In [358]:
print(os.listdir(rtdir))

['TCGA-FG-6689', 'TCGA-FG-7634', 'TCGA-CS-6188', 'TCGA-HT-7884', 'TCGA-DU-7015', 'TCGA-HT-7879', 'TCGA-DU-A5TU', 'TCGA-DU-A6S8', 'TCGA-CS-5393', 'TCGA-DU-8167', 'TCGA-DU-7306', 'TCGA-CS-4942', 'TCGA-CS-6186', 'TCGA-DU-8164', 'TCGA-HT-A61A', 'TCGA-HT-8111', 'TCGA-DU-7300', 'TCGA-DU-A5TY', 'TCGA-DU-7301', 'TCGA-HT-7686', 'TCGA-DU-6404', 'TCGA-DU-8162', 'TCGA-FG-5964', 'TCGA-DU-7299', 'TCGA-CS-6668', 'TCGA-DU-8168', 'TCGA-DU-5854', 'TCGA-CS-6666', 'TCGA-FG-6692', 'TCGA-HT-7680', 'license.html', 'TCGA-CS-5397', 'TCGA-HT-7690', 'TCGA-CS-4944', 'TCGA-DU-6542', 'TCGA-DU-8166', 'TCGA-DU-5872', 'TCGA-HT-8018', 'TCGA-CS-6665', 'TCGA-DU-7019', 'TCGA-DU-7008', 'TCGA-DU-A5TS', 'TCGA-CS-6669', 'TCGA-DU-A5TR', 'TCGA-FG-6691', 'TCGA-DU-7298', 'TCGA-HT-8114', 'TCGA-DU-7018', 'TCGA-DU-A5TT', 'TCGA-DU-A5TW', 'TCGA-DU-5855', 'TCGA-HT-7694', 'TCGA-HT-7473', 'TCGA-DU-5851', 'TCGA-DU-7010', 'TCGA-DU-5874', 'TCGA-DU-7309', 'TCGA-DU-7302', 'TCGA-HT-7602', 'TCGA-CS-5396', 'TCGA-DU-A6S7', 'TCGA-DU-7294', 'TCGA-D

In [359]:
print(os.listdir('/anvar/public_datasets/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG'))

['TCGA-CS-6188', 'TCGA-DU-6401', 'TCGA-DU-7015', 'TCGA-CS-5393', 'TCGA-DU-8167', 'TCGA-DU-7306', 'TCGA-CS-4942', 'TCGA-CS-6186', 'TCGA-DU-8164', 'TCGA-DU-A5TP', 'TCGA-DU-6402', 'TCGA-CS-5395', 'TCGA-DU-7300', 'TCGA-CS-6670', 'TCGA-DU-6407', 'TCGA-DU-5852', 'TCGA-DU-6400', 'TCGA-DU-7301', 'TCGA-DU-7013', 'TCGA-DU-6404', 'TCGA-DU-8162', 'TCGA-DU-7299', 'TCGA-CS-6668', 'TCGA-DU-8168', 'TCGA-DU-5854', 'TCGA-CS-6666', 'license.html', 'TCGA-CS-5397', 'TCGA-DU-6397', 'TCGA-CS-4944', 'TCGA-DU-6399', 'TCGA-DU-6408', 'TCGA-DU-6542', 'TCGA-DU-8166', 'TCGA-DU-5872', 'TCGA-DU-8165', 'TCGA-DU-5853', 'TCGA-CS-6665', 'TCGA-DU-7019', 'TCGA-DU-7008', 'TCGA-DU-8163', 'TCGA-CS-4938', 'TCGA-CS-4941', 'TCGA-CS-6669', 'TCGA-DU-7298', 'TCGA-DU-6405', 'TCGA-CS-5390', 'TCGA-CS-6290', 'TCGA-DU-7018', 'TCGA-DU-6410', 'TCGA-DU-5855', 'TCGA-DU-5851', 'TCGA-DU-7010', 'TCGA-DU-5874', 'TCGA-DU-7309', 'TCGA-CS-5394', 'TCGA-DU-5871', 'TCGA-DU-7302', 'TCGA-DU-5849', 'TCGA-CS-5396', 'TCGA-DU-7294', 'TCGA-DU-8158', 'TCGA-D

In [360]:
pd.Series(os.listdir(rtdir)).isin(
    os.listdir('/anvar/public_datasets/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG')
).sum()

39

In [379]:
list_subjs = list(
    pd.Series(os.listdir(rtdir))[
        pd.Series(os.listdir(rtdir)).isin(
    os.listdir('/anvar/public_datasets/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG')
)])

list_subjs =  list(set(list_subjs) - set(['license.html']))

### Ok here we go with the files:


1. Let's check the populating file, and print all folders (in segmentation and original data), and length of the folders.
2. Replace destinating in lgg_paths.csv

In [387]:
for i in list_subjs:
    print(
        glob.glob(
            '/anvar/public_datasets/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/' +i+'/*/*/')
    )

['/anvar/public_datasets/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-CS-6665/08-17-2001-MRI BRAIN WITH-50114/1201.000000-T1 COR SE-90930/', '/anvar/public_datasets/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-CS-6665/08-17-2001-MRI BRAIN WITH-50114/801.000000-PDWTSE-18378/', '/anvar/public_datasets/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-CS-6665/08-17-2001-MRI BRAIN WITH-50114/1001.000000-T1W3Dstryker-49008/', '/anvar/public_datasets/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-CS-6665/08-17-2001-MRI BRAIN WITH-50114/302.000000-dADC MAP-44456/', '/anvar/public_datasets/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-CS-6665/08-17-2001-MRI BRAIN WITH-50114/301.000000-DIFFUSION-36497/', '/anvar/public_datasets/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-CS-6665/08-17-2001-MRI BRAIN WITH-50114/601.000000-FFE FOR BLEED-92400/', '/anvar/public_datasets/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-CS-6665/08-17-

['/anvar/public_datasets/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-DU-7018/12-20-1991-MRI BRAIN WWO CONTRAST-98852/1101.000000-DTI ASSET-20961/', '/anvar/public_datasets/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-DU-7018/12-20-1991-MRI BRAIN WWO CONTRAST-98852/9.000000-AXIAL DIFFUSION-90314/', '/anvar/public_datasets/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-DU-7018/12-20-1991-MRI BRAIN WWO CONTRAST-98852/12.000000-AXIAL PERFUSION-99016/', '/anvar/public_datasets/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-DU-7018/12-20-1991-MRI BRAIN WWO CONTRAST-98852/1100.000000-DTI ASSET-99068/', '/anvar/public_datasets/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-DU-7018/12-20-1991-MRI BRAIN WWO CONTRAST-98852/8.000000-AXIAL FLAIR-28777/', '/anvar/public_datasets/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-DU-7018/12-20-1991-MRI BRAIN WWO CONTRAST-98852/13.000000-AX T1 POST GD FLAIR-47091/', '/anvar/public_datasets/tcia/ma

['/anvar/public_datasets/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-CS-5396/03-02-2001-MRI BRAIN CONTRAST MRA BRAIN-47605/403.000000-eBo-48146/', '/anvar/public_datasets/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-CS-5396/03-02-2001-MRI BRAIN CONTRAST MRA BRAIN-47605/901.000000-FFE FOR BLEED-67362/', '/anvar/public_datasets/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-CS-5396/03-02-2001-MRI BRAIN CONTRAST MRA BRAIN-47605/304.000000-LT COW-25699/', '/anvar/public_datasets/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-CS-5396/03-02-2001-MRI BRAIN CONTRAST MRA BRAIN-47605/1101.000000-T1 AX SE-54608/', '/anvar/public_datasets/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-CS-5396/03-02-2001-MRI BRAIN CONTRAST MRA BRAIN-47605/302.000000-3D MOTSA sense-85827/', '/anvar/public_datasets/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-CS-5396/03-02-2001-MRI BRAIN CONTRAST MRA BRAIN-47605/404.000000-eB1000i-30080/', '/anvar/public_da

'2.000000-SAG T1-14044'

In [441]:
t1_name.split('000000-')[-1][:-6]

'SAG T1'

In [442]:
for k in glob.glob('/anvar/public_datasets/tcia/manifest-1588284345182/TCGA-LGG/'+ i+ '/*/*/'):
        print(k.split('/')[-2], ',', len(os.listdir(k)))
        if t1_name.split('000000-')[-1][:-6] in k:
            print('hi')

300.000000-AXIAL FSE Glioma Segmentation Corrected-89966 , 1
300.000000-AXIAL T1 Glioma Segmentation Corrected-14702 , 1
300.000000-Ax FLAIR 2.5mm for surgery Glioma Segmentation Correc-05769 , 1
300.000000-COR T1 POST GD Glioma Segmentation-97477 , 1
300.000000-AXIAL FSE Glioma Segmentation-62868 , 1
300.000000-AXIAL T1 Glioma Segmentation-48897 , 1
300.000000-Ax FLAIR 2.5mm for surgery Glioma Segmentation-75393 , 1
300.000000-COR T1 POST GD Glioma Segmentation Corrected-34645 , 1


In [458]:
# I will print choosen files and find out if they are in segmentation folder by equal naming
count = 0

for i in list_subjs:
     
    print(i)
    print('------')
    t1_name = paths[paths['ID'] == i]['T1'].values[0].split('/')[-1]
    t2_name = paths[paths['ID'] == i]['T2'].values[0].split('/')[-1]
    fl_name = paths[paths['ID'] == i]['FLAIR'].values[0].split('/')[-1]
    ct1_name = paths[paths['ID'] == i]['T1C'].values[0].split('/')[-1] 
    
    print('Chosen folders', 'T1', paths[paths['ID'] == i]['T1'].values[0].split('/')[-1] )
    print('Chosen folders', 'T2', paths[paths['ID'] == i]['T2'].values[0].split('/')[-1] )
    print('Chosen folders', 'CT1', paths[paths['ID'] == i]['T1C'].values[0].split('/')[-1] )
    print('Chosen folders', 'FLAIR', paths[paths['ID'] == i]['FLAIR'].values[0].split('/')[-1] )
    
    
    print('------')
    for j in glob.glob('/anvar/public_datasets/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/'+ i+ '/*/*/'):
        print(j.split('/')[-2], ',', len(os.listdir(j)))
    print('------')    
    for k in glob.glob('/anvar/public_datasets/tcia/manifest-1588284345182/TCGA-LGG/'+ i+ '/*/*/'):
        print(ct1_name.split('000000-')[-1][:-6], k.split('/')[-2], ',', len(os.listdir(k)))
        if ct1_name.split('000000-')[-1][:-6] in k and 'Corrected' in k:
            print('! hi  !')
            count +=1

TCGA-CS-6665
------
Chosen folders T1 401.000000-T1W  SE-66214
Chosen folders T2 701.000000-T2WTSE-16485
Chosen folders CT1 1301.000000-T1 SAG SE-18213
Chosen folders FLAIR 501.000000-T2 AX FLAIR-26615
------
1201.000000-T1 COR SE-90930 , 22
801.000000-PDWTSE-18378 , 24
1001.000000-T1W3Dstryker-49008 , 200
302.000000-dADC MAP-44456 , 30
301.000000-DIFFUSION-36497 , 150
601.000000-FFE FOR BLEED-92400 , 30
1101.000000-T1 AX SE-32214 , 24
401.000000-T1W  SE-66214 , 16
304.000000-eB1000i-20523 , 30
1301.000000-T1 SAG SE-18213 , 16
901.000000-T1 AX  SE-68083 , 24
101.000000-Survey-25544 , 9
701.000000-T2WTSE-16485 , 24
501.000000-T2 AX FLAIR-26615 , 24
1003.000000-SAG MRP-26747 , 180
1002.000000-COR MPR-90692 , 200
303.000000-eBo-43475 , 30
------
T1 SAG SE 300.000000-T2WTSE Glioma Segmentation Corrected-99244 , 1
T1 SAG SE 300.000000-T1 AX  SE Glioma Segmentation Corrected-37705 , 1
T1 SAG SE 300.000000-T2 AX FLAIR Glioma Segmentation Corrected-85060 , 1
T1 SAG SE 300.000000-T1 AX SE Gliom

COR  T1 POST GD FLAIR 300.000000-AXIAL FLAIR Glioma Segmentation-17671 , 1
TCGA-DU-7294
------
Chosen folders T1 2.000000-SAG T1 FLAIR-59978
Chosen folders T2 219.000000-T2-weighted trace-66552
Chosen folders CT1 14.000000-AX T1 LOC  PODT GD-70066
Chosen folders FLAIR 7.000000-AXIAL FLAIR RF2  150-07121
------
7.000000-AXIAL FLAIR RF2  150-07121 , 68
222.000000-Dyz diffusion coefficient-62887 , 19
228.000000-Minimum eigenvalue-65612 , 19
230.000000-Maximum eigenvalue-41621 , 19
231.000000-Middle eigenvalue-19875 , 19
216.000000-Anisotropy index-08251 , 19
12.000000-AXIAL PERFUSION-22665 , 475
1.000000-3 PLANE LOC-67756 , 9
225.000000-Dzz diffusion coefficient-95384 , 19
220.000000-Dxy diffusion coefficient-16632 , 19
4.000000-AX T2 FR-FSE RF2 150-08103 , 34
3.000000-AX PD FSE-IR RF2 130-85826 , 34
224.000000-Surface DC-50108 , 19
219.000000-T2-weighted trace-66552 , 19
5.000000-AXIAL DIFFUSION-23727 , 114
213.000000-Surface  Average Aniso.-30471 , 19
208.000000-Average DC-40840 , 19
21

T1 SAG SE 300.000000-T1 AX  SE Glioma Segmentation Corrected-46437 , 1
T1 SAG SE 300.000000-T1 AX  SE Glioma Segmentation-63628 , 1
T1 SAG SE 300.000000-AXIAL  T2 Glioma Segmentation-85730 , 1
T1 SAG SE 300.000000-T2 AX FLAIR Glioma Segmentation Corrected-59124 , 1
T1 SAG SE 300.000000-T1W3DSTRYKER Glioma Segmentation-05362 , 1
T1 SAG SE 300.000000-AXIAL  T2 Glioma Segmentation Corrected-37412 , 1
T1 SAG SE 300.000000-T1W3DSTRYKER Glioma Segmentation Corrected-82481 , 1
TCGA-DU-7014
------
Chosen folders T1 7.000000-AXIAL T1 PRE SPECTROSCOPY-90412
Chosen folders T2 3.000000-AXIAL  FSE-84712
Chosen folders CT1 12.000000-AXIAL T1 POST GD THRU TUMOR-15984
Chosen folders FLAIR 5.000000-AXIAL FLAIR-06644
------
11.000000-AXIAL T1 POST GD-99296 , 5
7.000000-AXIAL T1 PRE SPECTROSCOPY-90412 , 4
8.000000-AX GRE POST SPECMODIFY CVS-08943 , 8
9.000000-AXIAL T1 PRE-GAD.-03736 , 60
10.000000-NEW AXIAL PERFUSION20S DELAY-51863 , 475
1.000000-SAG LOCAL-76105 , 18
2.000000-SAG LOCAL-98037 , 18
6.00000

In [459]:
count

8

In [341]:
os.listdir(rtdir + 'TCGA-DU-A5TS' + '/07-26-1997-MRI BRAIN WWO CONTRAST-00379')

['300.000000-AXIAL T1 Glioma Segmentation Corrected-55939',
 '300.000000-Ax FLAIR 2.5mm for surgery Glioma Segmentation-50760',
 '300.000000-AXIAL T1 Glioma Segmentation-75475',
 '300.000000-Ax T2 FSE Glioma Segmentation Corrected-47931',
 '300.000000-Ax T2 FSE Glioma Segmentation-20394',
 '300.000000-Ax T1 2.5mm for surgery Glioma Segmentation Corrected-29504',
 '300.000000-Ax T1 2.5mm for surgery Glioma Segmentation-47601',
 '300.000000-Ax FLAIR 2.5mm for surgery Glioma Segmentation Correc-90576']

In [174]:
for i in glob.glob('/anvar/public_datasets/tcia/TCGA*'):
    shutil.rmtree(i)

In [175]:
os.listdir('/anvar/public_datasets/tcia/')

['manifest-21CCx8fx964169794518489879',
 'manifest-1588284345182',
 'Pre-operative_TCGA_LGG_NIfTI_and_Segmentations',
 'brats_pipeline_out',
 'manifest-1614264588831',
 'manifest-25ZdsnMy5264093150469993200',
 'training_data',
 'egd',
 'manifest-nhlhmr7N4799206428618629939',
 'brats_toolkit_out',
 'gbm_dcm2niix_dataset',
 'Pre-operative_TCGA_GBM_NIfTI_and_Segmentations']

In [176]:
paths = pd.read_csv('../lgg_paths.csv')
basedir = '/anvar/public_datasets/tcia/'
savedir = '/anvar/public_datasets/preproc_study/lgg/dicom/'
# os.makedirs('/anvar/public_datasets/preproc_study/lgg/dicom/')

In [159]:
import shutil

for i in range(0, len(paths)):
    print(paths['ID'].iloc[i])
    try:
        os.mkdir(basedir + paths['ID'].iloc[i])
        shutil.copytree(basedir + paths['T1'].iloc[i][1:], savedir + paths['ID'].iloc[i] + '/T1/') 
        shutil.copytree(basedir + paths['T1C'].iloc[i][1:], savedir + paths['ID'].iloc[i] + '/CT1/') 
        shutil.copytree(basedir + paths['T2'].iloc[i][1:], savedir + paths['ID'].iloc[i] + '/T2') 
        shutil.copytree(basedir + paths['FLAIR'].iloc[i][1:], savedir + paths['ID'].iloc[i] + '/FLAIR/')
        
    except OSError as error:
        print(error) 
        pass  

TCGA-DU-7019
TCGA-DU-7302
TCGA-DU-5852
TCGA-DU-7014
TCGA-CS-6669
TCGA-DU-6405
TCGA-DU-5851
TCGA-DU-5854
TCGA-DU-6408
TCGA-DU-5874
TCGA-CS-5396
TCGA-DU-7306
TCGA-CS-5393
TCGA-DU-5849
TCGA-DU-6402
TCGA-CS-6670
TCGA-DU-5855
TCGA-DU-6407
TCGA-CS-4938
TCGA-CS-5390
TCGA-DU-7309
TCGA-CS-6186
TCGA-DU-7013
TCGA-DU-7015
TCGA-CS-5395
TCGA-CS-6666
TCGA-CS-4943
TCGA-CS-4941
TCGA-DU-8165
TCGA-CS-6667
TCGA-DU-5871
TCGA-DU-7010
TCGA-DU-6400
TCGA-DU-7300
TCGA-CS-6290
TCGA-DU-8164
TCGA-DU-7304
TCGA-DU-8166
TCGA-CS-6665
TCGA-DU-7301
TCGA-CS-4942
TCGA-DU-5872
TCGA-DU-7298
TCGA-DU-5853
TCGA-CS-5397
TCGA-CS-5394
TCGA-DU-7299
TCGA-CS-4944
TCGA-DU-6410
TCGA-DU-7294
TCGA-DU-8162
TCGA-CS-6668
TCGA-DU-7008
TCGA-DU-A5TP
TCGA-DU-6542
TCGA-DU-8158
TCGA-DU-8167
TCGA-CS-6188
TCGA-DU-8163
TCGA-DU-6401
TCGA-DU-6404
TCGA-DU-6397
TCGA-DU-6399
TCGA-DU-7018
TCGA-DU-8168
TCGA-DU-6395


In [228]:
os.listdir('/anvar/public_datasets/preproc_study/lgg/')

['dicom', 'orig']

#### Проверяем, что нифти файлы корректно сохранились и считаем, сколько их:

In [234]:
print ('T1', len(glob.glob('/anvar/public_datasets/preproc_study/lgg/orig/*/T1.nii.gz')))
print ('T2', len(glob.glob('/anvar/public_datasets/preproc_study/lgg/orig/*/T2.nii.gz')))
print ('FLAIR', len(glob.glob('/anvar/public_datasets/preproc_study/lgg/orig/*/FLAIR.nii.gz')))
print ('CT1', len(glob.glob('/anvar/public_datasets/preproc_study/lgg/orig/*/CT1.nii.gz')))

T1 66
T2 40
FLAIR 64
CT1 66


Опять потерялись T2, из за разного нейминга, попробуем сначала собрать ртструктуры, а потом будем чистить папку

#### Как найти нужную ртструктуру?

- Можно или сделать через поиск похожего нейминга, убрав цифры

- Или можно сделать через по series ID, как и было

In [200]:
os.listdir(rtdir + paths['ID'].iloc[1])

['12-03-1991-MRI BRAIN WWO CONTRAST-41440']

In [197]:
basedir = '/anvar/public_datasets/tcia/'

'/anvar/public_datasets/tcia/manifest-1588284345182/TCGA-LGG/'

In [198]:
os.listdir(basedir + paths['T1C'].iloc[i][1:])

['1-38.dcm',
 '1-36.dcm',
 '1-01.dcm',
 '1-16.dcm',
 '1-20.dcm',
 '1-03.dcm',
 '1-35.dcm',
 '1-34.dcm',
 '1-09.dcm',
 '1-14.dcm',
 '1-04.dcm',
 '1-12.dcm',
 '1-22.dcm',
 '1-06.dcm',
 '1-28.dcm',
 '1-17.dcm',
 '1-33.dcm',
 '1-05.dcm',
 '1-24.dcm',
 '1-39.dcm',
 '1-15.dcm',
 '1-31.dcm',
 '1-30.dcm',
 '1-32.dcm',
 '1-10.dcm',
 '1-41.dcm',
 '1-25.dcm',
 '1-18.dcm',
 '1-07.dcm',
 '1-29.dcm',
 '1-11.dcm',
 '1-08.dcm',
 '1-13.dcm',
 '1-27.dcm',
 '1-40.dcm',
 '1-21.dcm',
 '1-02.dcm',
 '1-19.dcm',
 '1-42.dcm',
 '1-26.dcm',
 '1-23.dcm',
 '1-37.dcm']

In [180]:
os.listdir('/anvar/public_datasets/tcia/manifest-1588284345182/TCGA-LGG/TCGA-DU-7302/12-03-1991-MRI BRAIN WWO CONTRAST-41440/')

['300.000000-AXIAL FLAIR Glioma Segmentation Corrected-95461',
 '300.000000-AXIAL FLAIR Glioma Segmentation-48843',
 '300.000000-AX T2 FR-FSE RF2 150 Glioma Segmentation-67602',
 '300.000000-AX T1 POST GD FLAIR Glioma Segmentation Corrected-46118',
 '300.000000-AX T2 FR-FSE RF2 150 Glioma Segmentation Corrected-36048',
 '300.000000-AX T1 pre gd Glioma Segmentation-10229',
 '300.000000-AX T1 POST GD FLAIR Glioma Segmentation-41682',
 '300.000000-AX T1 pre gd Glioma Segmentation Corrected-10339']

In [160]:
for i in range(0, len(paths)):
    print(paths['ID'].iloc[i])
    try:
#         os.mkdir(savedir + paths['ID'].iloc[i] + '/')
        print(glob.glob(rtdir + paths['ID'].iloc[i] +'/*/'))
        shutil.copytree(glob.glob(rtdir + paths['ID'].iloc[i] +'/*/')[0], savedir + paths['ID'].iloc[i] + '/RTSTRUCT/') 
      
    except Exception as error:
        print(error) 
        pass  

TCGA-DU-7019
['/anvar/public_datasets/tcia/manifest-1588284345182/TCGA-LGG/TCGA-DU-7019/09-08-1994-MRI BRAIN WWO CONTRAST-36813/']
TCGA-DU-7302
['/anvar/public_datasets/tcia/manifest-1588284345182/TCGA-LGG/TCGA-DU-7302/12-03-1991-MRI BRAIN WWO CONTRAST-41440/']
TCGA-DU-5852
[]
list index out of range
TCGA-DU-7014
['/anvar/public_datasets/tcia/manifest-1588284345182/TCGA-LGG/TCGA-DU-7014/06-18-1986-TUMOR VOL.- WAND-80617/']
TCGA-CS-6669
['/anvar/public_datasets/tcia/manifest-1588284345182/TCGA-LGG/TCGA-CS-6669/01-02-2002-MRI BRAIN COMBO-45633/']
TCGA-DU-6405
[]
list index out of range
TCGA-DU-5851
['/anvar/public_datasets/tcia/manifest-1588284345182/TCGA-LGG/TCGA-DU-5851/04-28-1995-MRI BRAIN WWO CONTRAST-02930/']
TCGA-DU-5854
['/anvar/public_datasets/tcia/manifest-1588284345182/TCGA-LGG/TCGA-DU-5854/11-04-1995-MRI BRAIN WWO CONTRAST-28000/']
TCGA-DU-6408
[]
list index out of range
TCGA-DU-5874
['/anvar/public_datasets/tcia/manifest-1588284345182/TCGA-LGG/TCGA-DU-5874/05-10-1995-MRI BRAI

In [162]:
len(glob.glob('/anvar/public_datasets/preproc_study/lgg/dicom/*/RTSTRUCT/'))

38

In [172]:
glob.glob('/anvar/public_datasets/preproc_study/lgg/dicom/*/RTSTRUCT/*/1-1.dcm')

['/anvar/public_datasets/preproc_study/lgg/dicom/TCGA-CS-6188/RTSTRUCT/300.000000-T1W  SE Glioma Segmentation-80190/1-1.dcm',
 '/anvar/public_datasets/preproc_study/lgg/dicom/TCGA-CS-6188/RTSTRUCT/300.000000-T2 AX FLAIR Glioma Segmentation Corrected-08132/1-1.dcm',
 '/anvar/public_datasets/preproc_study/lgg/dicom/TCGA-CS-6188/RTSTRUCT/300.000000-COR MPR Glioma Segmentation Corrected-80713/1-1.dcm',
 '/anvar/public_datasets/preproc_study/lgg/dicom/TCGA-CS-6188/RTSTRUCT/300.000000-T2WTSE Glioma Segmentation Corrected-41846/1-1.dcm',
 '/anvar/public_datasets/preproc_study/lgg/dicom/TCGA-CS-6188/RTSTRUCT/300.000000-T2WTSE Glioma Segmentation-94428/1-1.dcm',
 '/anvar/public_datasets/preproc_study/lgg/dicom/TCGA-CS-6188/RTSTRUCT/300.000000-T1W  SE Glioma Segmentation Corrected-37040/1-1.dcm',
 '/anvar/public_datasets/preproc_study/lgg/dicom/TCGA-CS-6188/RTSTRUCT/300.000000-T2 AX FLAIR Glioma Segmentation-73187/1-1.dcm',
 '/anvar/public_datasets/preproc_study/lgg/dicom/TCGA-CS-6188/RTSTRUCT/3

### Running script to compose *.nii images in folders

In [235]:
root = Path('/anvar/public_datasets/preproc_study/lgg/dicom')
df = join_tree(root, verbose=1)
df.to_csv(root.parent / 'meta.csv', index=False)
df = pd.read_csv(root.parent / 'meta.csv')

11703it [01:29, 221.37it/s]/home/kate/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (109,155,163,169) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
11711it [01:42, 221.37it/s]

In [236]:
paths = pd.read_csv('../lgg_paths.csv')
basedir = '/anvar/public_datasets/tcia/'
savedir = '/anvar/public_datasets/preproc_study/lgg/dicom/'

In [237]:
rtdir = '/anvar/public_datasets/tcia/manifest-1588284345182/TCGA-LGG/'

In [238]:
os.listdir(rtdir+'TCGA-FG-6689/03-26-2002-NR MRI BRAIN WWO-47983') 

['300.000000-MP RAGE AXIAL Glioma Segmentation Corrected-17638',
 '300.000000-MP RAGE AXIAL Glioma Segmentation-71597',
 '300.000000-T2 AXIALS Glioma Segmentation Corrected-93218',
 '300.000000-T2 AXIALS Glioma Segmentation-67348',
 '300.000000-MP RAGE AXIAL Glioma Segmentation-01663',
 '300.000000-MP RAGE AXIAL Glioma Segmentation Corrected-59980',
 '300.000000-FLAIR AXIALS Glioma Segmentation-48998',
 '300.000000-FLAIR AXIALS Glioma Segmentation Corrected-19647']

### Check *dcm files

In [258]:
def load_series(sub, root=None):
    """Loads DICOMs in specific order coincided with SegmentationStorage ordering."""
    if isinstance(root, Path):
        return order_series([dcmread(root / file.PathToFolder / file.FileName) for _, file in sub.iterrows()])[::-1]
    
def get_segmentation_storage_mask(segmentation_storage):
    """Returns (width, height, 3) mask, with values 1,2,3 corresponding to different BraTS classes."""
    mask = segmentation_storage.pixel_array
    mask = np.transpose(mask, (2,1,0))
    mask = np.array(np.split(mask, 3, axis=2))
    mask = np.array([1,2,3]).reshape((3, 1, 1, 1)) * mask
    return mask.sum(axis=0)

def get_reference_uid(segmentation_storage):
    """Returns reference SeriesInstanceUID of given SegmentationStorage"""
    return str(segmentation_storage.ReferencedSeriesSequence[0][('0020', '000e')].value)

def check_if_ct1_ref(df, rref):
    """Checks if input segmentation storage has CT1 image as reference."""
    sub = df.query('SeriesInstanceUID==@rref')
    return sub.PathToFolder.iloc[0].split('/')[1] == 'CT1'

In [124]:
list_seg = df.query('StudyInstanceUID == "1.3.6.1.4.1.14519.5.2.1.4591.4003.241850919885310339695393380617"').query('Modality == "SEG"')

In [138]:
list_seg['PathToFolder']

11369    TCGA-DU-7014/RTSTRUCT/300.000000-AXIAL T1 PRE-...
11370    TCGA-DU-7014/RTSTRUCT/300.000000-AXIAL T1 POST...
11371    TCGA-DU-7014/RTSTRUCT/300.000000-AXIAL  FSE Gl...
11372    TCGA-DU-7014/RTSTRUCT/300.000000-AXIAL T1 PRE-...
11373    TCGA-DU-7014/RTSTRUCT/300.000000-AXIAL FLAIR G...
11374    TCGA-DU-7014/RTSTRUCT/300.000000-AXIAL T1 POST...
11375    TCGA-DU-7014/RTSTRUCT/300.000000-AXIAL  FSE Gl...
11376    TCGA-DU-7014/RTSTRUCT/300.000000-AXIAL FLAIR G...
Name: PathToFolder, dtype: object

In [269]:
os.listdir( str(root) + '/TCGA-DU-7014/CT1')

['1-4.dcm', '1-1.dcm', '1-2.dcm', '1-3.dcm']

In [270]:
for i in range(8):
    print( 
        get_segmentation_storage_mask( 
            dcmread(str(root) + '/'+ list_seg['PathToFolder'].iloc[i] + '/1-1.dcm')
        ).shape
    )

(256, 256, 60)
(256, 256, 60)
(256, 256, 60)
(256, 256, 60)
(256, 256, 60)
(256, 256, 60)
(256, 256, 60)
(256, 256, 60)


In [79]:
df['SeriesInstanceUID'].value_counts()

1.3.6.1.4.1.14519.5.2.1.4591.4003.125577399578922720346879420953    325
1.3.6.1.4.1.14519.5.2.1.4591.4003.112592252232666023998953315710    180
1.3.6.1.4.1.14519.5.2.1.1188.4001.142459179818453226386988487183    170
1.3.6.1.4.1.14519.5.2.1.1188.4001.114078538471828186024243211217    170
1.3.6.1.4.1.14519.5.2.1.1188.4001.223094313796547802976465712442    170
                                                                   ... 
1.2.276.0.7230010.3.1.3.8323329.407669.1540933532.487109              1
1.2.276.0.7230010.3.1.3.8323329.408726.1540933534.642854              1
1.2.276.0.7230010.3.1.3.8323329.410577.1540933539.503835              1
1.2.276.0.7230010.3.1.3.8323329.408738.1540933534.634701              1
1.2.276.0.7230010.3.1.3.8323329.423577.1540933574.890161              1
Name: SeriesInstanceUID, Length: 568, dtype: int64

In [112]:
df.query('StudyInstanceUID == "1.3.6.1.4.1.14519.5.2.1.4591.4003.156635755638398578615698188040"')

,AccessionNumber,AcquisitionDate,AcquisitionNumber,AcquisitionTime,AdmittingDiagnosesDescription,BitsAllocated,BitsStored,BodyPartExamined,BurnedInAnnotation,CodeMeaning,...,InStackPositionNumber,StackID,StudyComments,EthnicGroup,StudyArrivalDate,StudyArrivalTime,StudyCompletionDate,StudyCompletionTime,StudyStatusID,PatientOrientation


In [263]:
def get_segmentation_storage_mask(segmentation_storage):
    """Returns (width, height, 3) mask, with values 1,2,3 corresponding to different BraTS classes."""
    mask = segmentation_storage.pixel_array
    mask = np.transpose(mask, (2,1,0))
    mask = np.array(np.split(mask, 3, axis=2))
    mask = np.array([1,2,3]).reshape((3, 1, 1, 1)) * mask
    return mask.sum(axis=0)

get_segmentation_storage_mask(segmentation_storage)

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [ ]:
## поменять название рт структур

In [260]:
segmentation_storage

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 200
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: Segmentation Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.2.276.0.7230010.3.1.4.8323329.414664.1540933547.687260
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.3.6.1.4.1.22213.1.143
(0002, 0013) Implementation Version Name         SH: '0.5'
(0002, 0016) Source Application Entity Title     AE: 'POSDA'
-------------------------------------------------
(0008, 0008) Image Type                          CS: ['DERIVED', 'PRIMARY']
(0008, 0016) SOP Class UID                       UI: Segmentation Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.276.0.7230010.3.1.4.8323329.414664.1540933547.687260
(0008, 0020) Study Date                          DA: '1986

In [295]:
for i in glob.glob('/anvar/public_datasets/preproc_study/lgg/dicom/*/RTSTRUCT_DICOM/'):
#     print(len(os.listdir(i)))
#     shutil.rmtree(i)

In [279]:
os.listdir('/anvar/public_datasets/preproc_study/lgg/dicom/TCGA-CS-5393')

['T1', 'T2', 'CT1', 'RTSTRUCT', 'FLAIR']

In [280]:
# # Only save CT1 (=T1c) mask, `Corrected`

for patient_folder in tqdm(root.glob('*')): # 1
    if any('RTSTRUCT' in subfolders.name for subfolders in patient_folder.glob('*')):
        rtstruct_dicom_folder = patient_folder / 'RTSTRUCT_DICOM'
        if not rtstruct_dicom_folder.exists(): # 2
            rtstruct_dicom_folder.mkdir()
        (rtstruct_dicom_folder / 'CT1').mkdir()

66it [00:00, 13969.72it/s]


### В лоб получается только 2 пациента:

In [294]:
for patient_folder in tqdm(root.glob('*')): 
    rtstruct_dicom_folder = patient_folder / 'RTSTRUCT_DICOM' / 'CT1'
    try:
        for rtstruct_subfolder in (patient_folder / 'RTSTRUCT').glob('*'):
            segmentation_storage = dcmread(rtstruct_subfolder / '1-1.dcm')
            reference_uid = get_reference_uid(segmentation_storage)
#             print(patient_folder.name,reference_uid)
#             if (check_if_ct1_ref(df, reference_uid) and
#                (('Correc' in rtstruct_subfolder.name) or
#                  (patient_folder.name in no_corrected))): # Mask is for CT1 and corrected
            sub = df.query('SeriesInstanceUID==@reference_uid')
            image_series = load_series(sub, root)
            mask = get_segmentation_storage_mask(segmentation_storage)
            mask_to_dicom(image_series, mask, folder=rtstruct_dicom_folder)  
            print(rtstruct_subfolder)
    except Exception as e:
        print(e, patient_folder.name)

0it [00:00, ?it/s]

/anvar/public_datasets/preproc_study/lgg/dicom/TCGA-CS-6188/RTSTRUCT/300.000000-T1W  SE Glioma Segmentation-80190
/anvar/public_datasets/preproc_study/lgg/dicom/TCGA-CS-6188/RTSTRUCT/300.000000-T2 AX FLAIR Glioma Segmentation Corrected-08132


1it [00:01,  1.14s/it]

not enough values to unpack (expected 1, got 0) TCGA-CS-6188
/anvar/public_datasets/preproc_study/lgg/dicom/TCGA-DU-7015/RTSTRUCT/300.000000-AX T1 POST GD FLAIR Glioma Segmentation Corrected-94531
/anvar/public_datasets/preproc_study/lgg/dicom/TCGA-DU-7015/RTSTRUCT/300.000000-AX T1 POST GD FLAIR Glioma Segmentation-67010
/anvar/public_datasets/preproc_study/lgg/dicom/TCGA-DU-7015/RTSTRUCT/300.000000-AX T2 FR-FSE RF2 150 Glioma Segmentation Corrected-21243


3it [00:07,  1.80s/it]

...

/anvar/public_datasets/preproc_study/lgg/dicom/TCGA-DU-7302/RTSTRUCT/300.000000-AX T2 FR-FSE RF2 150 Glioma Segmentation-67602
not enough values to unpack (expected 1, got 0) TCGA-DU-7302


62it [00:53,  1.90it/s]

not enough values to unpack (expected 1, got 0) TCGA-CS-5396
not enough values to unpack (expected 1, got 0) TCGA-DU-7294
not enough values to unpack (expected 1, got 0) TCGA-DU-7304


66it [00:53,  1.23it/s]

not enough values to unpack (expected 1, got 0) TCGA-DU-7014


#### Проверим, для скольких папок с разметкой не хватает её в дикомах: 

Оказывается больше, чем для половины

In [298]:
for patient_folder in tqdm(root.glob('*')): 
    try:
        if len(os.listdir(patient_folder / 'RTSTRUCT_DICOM' / 'CT1')) > 0:
            print('hi')
    except:
        pass

66it [00:00, 45418.22it/s]


#### Сделаем список из нужных 38 дикомов и их стравим:

In [311]:
os.listdir(rtdir +'TCGA-DU-7019'+ '/09-08-1994-MRI BRAIN WWO CONTRAST-36813/' )

['300.000000-AX T1 pre gd Glioma Segmentation-44351',
 '300.000000-AX T2 FR-FSE RF2 150 Glioma Segmentation Corrected-83452',
 '300.000000-AX T1 pre gd Glioma Segmentation Corrected-39983',
 '300.000000-AX T1 POST GD FLAIR Glioma Segmentation Corrected-02719',
 '300.000000-AXIAL FLAIR Glioma Segmentation-35927',
 '300.000000-AX T1 POST GD FLAIR Glioma Segmentation-92884',
 '300.000000-AX T2 FR-FSE RF2 150 Glioma Segmentation-69613',
 '300.000000-AXIAL FLAIR Glioma Segmentation Corrected-38412']

In [312]:
os.listdir(rtdir +'TCGA-DU-7019'+ '/09-08-1994-MRI BRAIN WWO CONTRAST-36813/' )

PosixPath('/anvar/public_datasets/preproc_study/lgg/dicom')

In [325]:
paths['T1C'][9]

' manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-DU-5874/05-10-1995-MRI BRAIN WWO CONTRAST-73527/13.000000-COR  T1 POST GD FLAIR-85867'

In [330]:
os.listdir('/anvar/public_datasets/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-DU-5874/05-10-1995-MRI BRAIN WWO CONTRAST-73527/13')

FileNotFoundError: [Errno 2] No such file or directory: '/anvar/public_datasets/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-DU-5874/05-10-1995-MRI BRAIN WWO CONTRAST-73527/13'

In [314]:
os.listdir('/anvar/public_datasets/tcia/manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-DU-7019/09-08-1994-MRI BRAIN WWO CONTRAST-36813/')

['11.000000-AX T1 POST GD FLAIR-34098',
 '8.000000-AX T1 pre gd-46981',
 '10.000000-3D DCE DYNAMIC PERF-58380',
 '9.000000-3D DCE T1 MAPPING X 5-16194',
 '7.000000-DTI 2.6mm WH-58503',
 '702.000000-DTI 2.6mm WH-19610',
 '700.000000-DTI 2.6mm WH-17852',
 '2.000000-ASSET cal-08217',
 '3.000000-SAG T1 FLAIR-41565',
 '5.000000-AX T2 FR-FSE RF2 150-56476',
 '12.000000-COR  T1 POST GD FLAIR-11229',
 '4.000000-AXIAL DIFFUSION-73794',
 '701.000000-DTI 2.6mm WH-37159',
 '1.000000-3 PLANE LOC-70172',
 '6.000000-AXIAL FLAIR-36919']

In [303]:
for i in range(0, len(paths)):
    print(paths['ID'].iloc[i])
    print(paths['T1C'].iloc[i][1:])

TCGA-DU-7019
manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-DU-7019/09-08-1994-MRI BRAIN WWO CONTRAST-36813/12.000000-COR  T1 POST GD FLAIR-11229
TCGA-DU-7302
manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-DU-7302/12-03-1991-MRI BRAIN WWO CONTRAST-41440/12.000000-COR  T1 POST GD FLAIR-19937
TCGA-DU-5852
manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-DU-5852/07-09-1995-MRI BRAIN WWO CONTRAST-62702/12.000000-COR  T1 POST GD FLAIR-51539
TCGA-DU-7014
manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-DU-7014/06-18-1986-TUMOR VOL.- WAND-80617/12.000000-AXIAL T1 POST GD THRU TUMOR-15984
TCGA-CS-6669
manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-CS-6669/01-02-2002-MRI BRAIN COMBO-45633/1101.000000-T1 SAG SE-52125
TCGA-DU-6405
manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-DU-6405/10-05-1985-TUMOR VOL-13206/10.000000-AXIAL T1 POST GD-52534
TCGA-DU-5851
manifest-25ZdsnMy5264093150469993200/TCGA-LGG/TCGA-DU-5851/04-28-1995-MRI BRAIN WWO CONTRAST-02930/9.000000-COR T1 POST GD

In [ ]:
for i in range(0, len(paths)):
    print(paths['ID'].iloc[i])
    try:
#         os.mkdir(savedir + paths['ID'].iloc[i] + '/')
        print(glob.glob(rtdir + paths['ID'].iloc[i] +'/*/'))
#         shutil.copytree(glob.glob(rtdir + paths['ID'].iloc[i] +'/*/')[0], savedir + paths['ID'].iloc[i] + '/RTSTRUCT/') 
      
    except Exception as error:
        print(error) 
        pass  

## Checking if there are enough folders with RTstruct

In [282]:
len(glob.glob('/anvar/public_datasets/preproc_study/lgg/dicom/*/RTSTRUCT_DICOM/CT1/' ))

38

In [114]:
patient_folder

PosixPath('/anvar/public_datasets/preproc_study/lgg/dicom/TCGA-DU-6395')

In [285]:
os.listdir('/anvar/public_datasets/preproc_study/lgg/dicom/TCGA-DU-7014/RTSTRUCT_DICOM/CT1')

[]

In [91]:
from dicom_csv.misc import stack_images

def load_series_2(path):
    """Loads DICOMs in specific order coincided with SegmentationStorage ordering."""
    return order_series([dcmread(filename) for filename in path.glob('*')])

In [ ]:
# im1 = stack_images(load_series_2(root / 'TCGA-06-0162' / 'CT1'))
# mask1 = stack_images(load_series_2(root / 'TCGA-06-0162' / 'RTSTRUCT_DICOM' / 'CT1'))